In [1]:
import wrds
import pandas as pd
import numpy as np

In [3]:
conn = wrds.Connection(wrds_username='vidhiagrawal09')

Loading library list...
Done


### Data Collection

#### Top 800 Companies data with key financial variables

In [7]:
query_q="""WITH last_trading_days AS (
    SELECT
        DATE_TRUNC('quarter', dlycaldt) AS quarter,
        MAX(dlycaldt) AS last_date
    FROM crsp_a_stock.wrds_dsfv2_query
    WHERE dlycaldt BETWEEN '2013-01-01' AND '2023-12-31'
    GROUP BY DATE_TRUNC('quarter', dlycaldt)
),

ranked_companies AS (
    SELECT DISTINCT
        a.dlycaldt, a.dlycap, a.ticker, a.dlyprc, a.dlyprevprc, a.permno,
        ROW_NUMBER() OVER (PARTITION BY DATE_TRUNC('quarter', a.dlycaldt) ORDER BY a.dlycap DESC) AS rank
    FROM crsp_a_stock.wrds_dsfv2_query a
    JOIN last_trading_days b ON a.dlycaldt = b.last_date
    WHERE a.dlycap IS NOT NULL
),

top_600_companies AS (
    SELECT permno, dlycaldt, dlycap, ticker, dlyprc
    FROM ranked_companies
    WHERE rank <= 800
),

crsp_data AS (
    SELECT
        permno,
        dlycaldt,
        AVG(dlyvol) OVER (PARTITION BY permno ORDER BY dlycaldt ROWS BETWEEN 90 PRECEDING AND CURRENT ROW) AS avg_volume_3m,
        dlyprc AS stock_price,
        ((dlyprc - dlyprevprc) / NULLIF(dlyprevprc, 0)) * 100 AS return_1m
    FROM crsp_a_stock.wrds_dsfv2_query
    WHERE dlycaldt BETWEEN '2013-01-01' AND '2023-12-31'
    AND dlycap IS NOT NULL
),

crsp_ccm AS (
    SELECT
        gvkey,
        lpermno AS permno,
        linkdt,
        linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LC', 'LU', 'LS')
    AND usedflag = 1
),

compustat_fundamentals AS (
    SELECT 
        gvkey,
        datadate,
        fyearq,
        fqtr,
        -- Use COALESCE to handle potential NULL values
        COALESCE(atq, atq_lags_1, atq_lags_2) AS total_assets,
        COALESCE(ltq, ltq_lags_1, ltq_lags_2) AS total_liabilities,
        COALESCE(niq, niq_lags_1, niq_lags_2) AS net_income,
        COALESCE(oibdpq, oibdpq_lags_1, oibdpq_lags_2) AS operating_income,
        COALESCE(subquery.oancfy, oancfq_lags_1, oancfq_lags_2) AS cash_flow_from_operations,
        
        -- Careful calculations with NULLIF to avoid division by zero
        CASE 
            WHEN COALESCE(actq, 0) > 0 AND COALESCE(lctq, 0) > 0 
            THEN actq / lctq 
            ELSE NULL 
        END AS current_ratio,
        
        CASE 
            WHEN COALESCE(ltq, 0) > 0 AND COALESCE(seqq, 0) > 0 
            THEN ltq / seqq 
            ELSE NULL 
        END AS debt_to_equity,
        
        CASE 
            WHEN COALESCE(atq, 0) > 0 
            THEN niq / atq 
            ELSE NULL 
        END AS roa,
        
        CASE 
            WHEN COALESCE(seqq, 0) > 0 
            THEN niq / seqq 
            ELSE NULL 
        END AS roe,
        
        COALESCE(epspiq, epspiq_lags_1, epspiq_lags_2) AS eps,
        
        CASE 
            WHEN COALESCE(seqq, 0) > 0 AND COALESCE(cshoq, 0) > 0 
            THEN seqq / cshoq 
            ELSE NULL 
        END AS book_value_per_share,
        
        cik
    FROM (
        SELECT 
            *,
            LAG(atq, 1) OVER (PARTITION BY gvkey ORDER BY datadate) AS atq_lags_1,
            LAG(atq, 2) OVER (PARTITION BY gvkey ORDER BY datadate) AS atq_lags_2,
            LAG(ltq, 1) OVER (PARTITION BY gvkey ORDER BY datadate) AS ltq_lags_1,
            LAG(ltq, 2) OVER (PARTITION BY gvkey ORDER BY datadate) AS ltq_lags_2,
            LAG(niq, 1) OVER (PARTITION BY gvkey ORDER BY datadate) AS niq_lags_1,
            LAG(niq, 2) OVER (PARTITION BY gvkey ORDER BY datadate) AS niq_lags_2,
            LAG(oibdpq, 1) OVER (PARTITION BY gvkey ORDER BY datadate) AS oibdpq_lags_1,
            LAG(oibdpq, 2) OVER (PARTITION BY gvkey ORDER BY datadate) AS oibdpq_lags_2,
            LAG(fundq.oancfy, 1) OVER (PARTITION BY gvkey ORDER BY datadate) AS oancfq_lags_1,
            LAG(fundq.oancfy, 2) OVER (PARTITION BY gvkey ORDER BY datadate) AS oancfq_lags_2,
            LAG(epspiq, 1) OVER (PARTITION BY gvkey ORDER BY datadate) AS epspiq_lags_1,
            LAG(epspiq, 2) OVER (PARTITION BY gvkey ORDER BY datadate) AS epspiq_lags_2
        FROM comp.fundq
        WHERE datadate BETWEEN '2013-01-01' AND '2023-12-31'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'
        AND popsrc = 'D'
        AND consol = 'C'
    ) AS subquery
),

ibes_data AS (
    SELECT
        ticker,
        statpers,
        COUNT(DISTINCT numest) AS num_analysts_covering
    FROM ibes.statsum_epsus
    WHERE statpers BETWEEN '2013-01-01' AND '2023-12-31'
    GROUP BY ticker, statpers
),

auditor_changes AS (
    SELECT
        ac.company_fkey,
        COUNT(*) AS num_auditor_changes
    FROM audit_audit_comp.feed02_auditor_changes ac
    WHERE ac.file_date BETWEEN '2013-01-01' AND '2023-12-31'
    GROUP BY ac.company_fkey
),

financial_restatements AS (
    SELECT
        company_fkey,
        COUNT(*) AS num_restatements
    FROM audit_audit_comp.feed39_financial_restatements
    WHERE file_date BETWEEN '2013-01-01' AND '2023-12-31'
    GROUP BY company_fkey
)

-- Final merged query with more robust joining
SELECT
    cd.permno,
    cd.dlycaldt,
    cd.avg_volume_3m,
    cd.stock_price,
    cd.return_1m,
    tc.dlycap,
    tc.ticker,
    (SELECT DISTINCT FIRST_VALUE(ch.hsic) OVER (PARTITION BY ch.gvkey ORDER BY cf.datadate DESC)) as hsic,
    (SELECT DISTINCT FIRST_VALUE(ch.hgind) OVER (PARTITION BY ch.gvkey ORDER BY cf.datadate DESC)) as hgind,
    cf.total_assets,
    cf.total_liabilities,
    cf.net_income,
    cf.operating_income,
    cf.cash_flow_from_operations,
    cf.current_ratio,
    cf.debt_to_equity,
    cf.roa,
    cf.roe,
    cf.eps,
    cf.book_value_per_share,
    COALESCE(i.num_analysts_covering, 0) AS num_analysts_covering,
    COALESCE(ac.num_auditor_changes, 0) AS num_auditor_changes,
    COALESCE(fr.num_restatements, 0) AS num_restatements
FROM crsp_data cd
JOIN top_600_companies tc ON cd.permno = tc.permno
    AND cd.dlycaldt = tc.dlycaldt
JOIN crsp_ccm ccm ON cd.permno = ccm.permno
    AND cd.dlycaldt >= ccm.linkdt
    AND (cd.dlycaldt <= ccm.linkenddt OR ccm.linkenddt IS NULL)
JOIN compustat_fundamentals cf ON ccm.gvkey = cf.gvkey
    -- More flexible date matching using quarters
    AND DATE_TRUNC('quarter', cd.dlycaldt) = DATE_TRUNC('quarter', cf.datadate)
LEFT JOIN crsp_a_ccm.comphist ch ON ccm.gvkey = ch.gvkey
LEFT JOIN ibes_data i ON tc.ticker = i.ticker
    AND DATE_TRUNC('month', cd.dlycaldt) = DATE_TRUNC('month', i.statpers)
LEFT JOIN auditor_changes ac ON cf.cik = ac.company_fkey
LEFT JOIN financial_restatements fr ON cf.cik = fr.company_fkey
"""


def get_industry_name(sic_code):
    sic_str = str(int(sic_code))
    sic_4digit = int(sic_str) if len(sic_str) >= 4 else 0
    sic_3digit = int(sic_str[:3]) if len(sic_str) >= 3 else 0
    sic_2digit = int(sic_str[:2]) if len(sic_str) >= 2 else 0

    tech_sic_codes = {
        # Traditional Tech
        357: "Technology",    # Computer Hardware (e.g., Apple)
        737: "Technology",    # Software & Services (e.g., Microsoft)
        367: "Technology",    # Semiconductors
        366: "Technology",    # Communications Equipment

        # Internet & Digital Services
        5961: "Technology",   # E-commerce (e.g., Amazon)
        7375: "Technology",   # Information Retrieval Services (e.g., Google)
        7374: "Technology",   # Computer Processing (e.g., Meta/Facebook)
        7372: "Technology",   # Software Publishers
        4899: "Technology",   # Communications Services (e.g., Netflix)

        # Additional Tech Manufacturing
        3674: "Technology",   # Semiconductors & Related Devices
        3571: "Technology",   # Electronic Computers
        3577: "Technology",   # Computer Peripheral Equipment
    }

    # 4-digit SIC
    if sic_4digit in tech_sic_codes:
        return tech_sic_codes[sic_4digit]

    #  3-digit SIC
    if sic_3digit in tech_sic_codes:
        return tech_sic_codes[sic_3digit]

    # General industry classifications
    industry_mapping = {
        range(1, 10): "Agriculture, Forestry, Fishing",
        range(10, 15): "Mining",
        range(15, 18): "Construction",
        range(20, 40): "Manufacturing",
        range(40, 50): "Transportation & Public Utilities",
        range(50, 52): "Wholesale Trade",
        range(52, 60): "Retail Trade",
        range(60, 68): "Finance, Insurance, Real Estate",
        range(70, 90): "Services",
        range(90, 100): "Public Administration"
    }

    for sic_range, industry in industry_mapping.items():
        if sic_2digit in sic_range:
            return industry
    return "Unknown"

In [9]:
final_data= conn.raw_sql(query_q)
final_data= final_data.drop_duplicates(subset=['permno','dlycaldt'],keep='first')
final_data.loc[:, 'industry_name'] = final_data['hsic'].apply(get_industry_name)

In [11]:
final_data.groupby(['dlycaldt']).size()

dlycaldt
2013-03-28    799
2013-06-28    800
2013-09-30    798
2013-12-31    798
2014-03-31    799
2014-06-30    799
2014-09-30    800
2014-12-31    798
2015-03-31    798
2015-06-30    796
2015-09-30    797
2015-12-31    794
2016-03-31    796
2016-06-30    796
2016-09-30    796
2016-12-30    795
2017-03-31    795
2017-06-30    794
2017-09-29    794
2017-12-29    795
2018-03-29    794
2018-06-29    795
2018-09-28    794
2018-12-31    793
2019-03-29    794
2019-06-28    793
2019-09-30    795
2019-12-31    794
2020-03-31    792
2020-06-30    794
2020-09-30    793
2020-12-31    796
2021-03-31    795
2021-06-30    796
2021-09-30    796
2021-12-31    795
2022-03-31    796
2022-06-30    797
2022-09-30    792
2022-12-30    797
2023-03-31    796
2023-06-30    796
2023-09-29    795
2023-12-29    797
dtype: int64

#### S&P Quarterly Data Collection

In [14]:
import pandas as pd

# Fetch S&P 500 membership data with daily returns
sp500 = conn.raw_sql("""
    SELECT a.*, b.date, b.ret
    FROM crsp.dsp500list AS a
    JOIN crsp.dsf AS b
    ON a.permno = b.permno
    WHERE b.date >= a.start AND b.date <= a.ending
    AND b.date >= '2013-01-01' AND b.date <= '2023-12-31'
    ORDER BY b.date;
""", date_cols=['start', 'ending', 'date'])

# Identify the last trading day of each quarter
sp500['quarter'] = sp500['date'].dt.to_period('Q')
last_trading_days = sp500.groupby('quarter')['date'].max().reset_index()

# Filter S&P 500 data for the last trading day of each quarter
sp500_quarterly = pd.merge(sp500, last_trading_days, on='date')

# Add other identifiers from CRSP.MSENAMES
mse = conn.raw_sql("""
    SELECT comnam, ncusip, namedt, nameendt, 
           permno, shrcd, exchcd, hsiccd, ticker
    FROM crsp.msenames
""", date_cols=['namedt', 'nameendt'])

mse['nameendt'] = mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with S&P 500 data
sp500_quarterly_full = pd.merge(sp500_quarterly, mse, how='left', on='permno')
sp500_quarterly_full = sp500_quarterly_full.loc[
    (sp500_quarterly_full['date'] >= sp500_quarterly_full['namedt']) &
    (sp500_quarterly_full['date'] <= sp500_quarterly_full['nameendt'])
]

# Add Compustat Identifiers using CCM Linking Table
ccm = conn.raw_sql("""
    SELECT gvkey, liid AS iid, lpermno AS permno,
           linktype, linkprim, linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE SUBSTR(linktype, 1, 1) = 'L'
    AND (linkprim = 'C' OR linkprim = 'P')
""", date_cols=['linkdt', 'linkenddt'])

ccm['linkenddt'] = ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Merge with S&P 500 quarterly data
sp500_quarterly_ccm = pd.merge(sp500_quarterly_full, ccm, how='left', on=['permno'])
sp500_quarterly_ccm = sp500_quarterly_ccm.loc[
    (sp500_quarterly_ccm['date'] >= sp500_quarterly_ccm['linkdt']) &
    (sp500_quarterly_ccm['date'] <= sp500_quarterly_ccm['linkenddt'])
]

# Drop unnecessary columns and rearrange for final output
sp500_quarterly_ccm = sp500_quarterly_ccm.drop(columns=[
    'namedt', 'nameendt', 'linktype', 'linkprim', 'linkdt', 'linkenddt'
])
sp500_quarterly_ccm = sp500_quarterly_ccm[[
    'date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd',
    'ticker', 'gvkey', 'iid', 'start', 'ending', 'ret'
]]

# Add CIKs and link with SEC Index Files using CIK
names = conn.raw_sql("""
    SELECT gvkey, cik, sic, naics, gind, gsubind
    FROM comp.names
""")
sp500_quarterly_cik = pd.merge(sp500_quarterly_ccm, names, on='gvkey', how='left')

# Display the final table
sp500_quarterly_cik.head()


,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret,cik,sic,naics,gind,gsubind
0,2013-03-28,79881,URBAN OUTFITTERS INC,91704710,11,3,5650,URBN,029150,01,2010-02-08,2017-03-17,-0.010220,0000912615,5651,458110,255040,25504010
1,2013-03-28,23819,HALLIBURTON CO,40621610,11,1,1389,HAL,005439,01,1957-03-01,2023-12-29,-0.006149,0000045012,1389,213112,101010,10101020
2,2013-03-28,14277,SCHLUMBERGER LTD,80685710,12,1,5651,SLB,009465,01,1965-03-04,2023-12-29,-0.002796,0000087347,1389,213112,101010,10101020
3,2013-03-28,59010,GAP INC,36476010,11,1,5651,GPS,004990,01,1986-08-21,2022-02-02,0.003686,0000039911,5651,458110,255040,25504010
4,2013-03-28,43449,MCDONALDS CORP,58013510,11,1,5812,MCD,007154,01,1970-06-25,2023-12-29,0.007988,0000063908,5812,722513,253010,25301040


In [15]:
sp500_quarterly_cik.groupby(['date']).size()

date
2013-03-28    501
2013-06-28    501
2013-09-30    501
2013-12-31    501
2014-03-31    501
2014-06-30    501
2014-09-30    501
2014-12-31    501
2015-03-31    501
2015-06-30    500
2015-09-30    501
2015-12-31    501
2016-03-31    501
2016-06-30    501
2016-09-30    501
2016-12-30    501
2017-03-31    501
2017-06-30    501
2017-09-29    501
2017-12-29    500
2018-03-29    501
2018-06-29    501
2018-09-28    501
2018-12-31    501
2019-03-29    501
2019-06-28    501
2019-09-30    501
2019-12-31    501
2020-03-31    501
2020-06-30    501
2020-09-30    501
2020-12-31    501
2021-03-31    500
2021-06-30    500
2021-09-30    500
2021-12-31    500
2022-03-31    500
2022-06-30    500
2022-09-30    499
2022-12-30    500
2023-03-31    500
2023-06-30    500
2023-09-29    500
2023-12-29    500
dtype: int64

In [16]:
sp500_quarterly_cik = sp500_quarterly_cik.drop_duplicates(subset=['permno','date'],keep='first')
sp500_quarterly_cik['In_S&P']=1
sp500_quarterly_cik.groupby(['date']).size()

date
2013-03-28    500
2013-06-28    500
2013-09-30    500
2013-12-31    500
2014-03-31    500
2014-06-30    500
2014-09-30    500
2014-12-31    500
2015-03-31    500
2015-06-30    499
2015-09-30    500
2015-12-31    500
2016-03-31    500
2016-06-30    500
2016-09-30    500
2016-12-30    500
2017-03-31    500
2017-06-30    500
2017-09-29    500
2017-12-29    499
2018-03-29    500
2018-06-29    500
2018-09-28    500
2018-12-31    500
2019-03-29    500
2019-06-28    500
2019-09-30    500
2019-12-31    500
2020-03-31    500
2020-06-30    500
2020-09-30    500
2020-12-31    500
2021-03-31    500
2021-06-30    500
2021-09-30    500
2021-12-31    500
2022-03-31    500
2022-06-30    500
2022-09-30    499
2022-12-30    500
2023-03-31    500
2023-06-30    500
2023-09-29    500
2023-12-29    500
dtype: int64

### Join S&P and Top 800 Companies data

In [21]:
import pandas as pd

# Load your datasets (assuming you have already imported them as `top600` and `sp500`)
# Ensure date columns are datetime
final_data['dlycaldt'] = pd.to_datetime(final_data['dlycaldt'])
sp500_quarterly_cik['date'] = pd.to_datetime(sp500_quarterly_cik['date'])

# Perform a left join on 'permno' and 'dlycaldt'/'date'
merged = pd.merge(final_data, sp500_quarterly_cik, 
                  left_on=['permno', 'dlycaldt'], 
                  right_on=['permno', 'date'], 
                  how='left')

# Add a binary column 'in_sp500'
merged['in_sp500'] = merged['In_S&P'].notna().astype(int)

# # Drop unnecessary columns from the S&P data (optional)
columns_to_drop = ['In_S&P', 'date', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 
                   'gvkey', 'iid', 'start', 'ending', 'ret', 'cik', 'sic', 'naics', 
                   'gind', 'gsubind','hsic','hgind']
merged = merged.drop(columns=columns_to_drop, errors='ignore')

# Save or inspect the merged dataset
print(merged.head())

   permno   dlycaldt  avg_volume_3m  stock_price  return_1m       dlycap  \
0   87432 2013-03-28   3.477558e+06        41.97  -0.285103  14570263.23   
1   87432 2013-06-28   3.722482e+06        42.76  -0.742804  14729793.76   
2   87432 2013-09-30   2.904071e+06        51.25  -0.736006  16953090.00   
3   87432 2013-12-31   2.475560e+06        57.19  -0.226797  18976156.71   
4   87432 2014-03-31   2.614428e+06        55.92   2.249040  18644902.32   

  ticker_x  total_assets  total_liabilities  net_income  ...       roa  \
0        A       10653.0             5302.0       179.0  ...  0.016803   
1        A       10587.0             5279.0       166.0  ...  0.015680   
2        A       10278.0             5488.0       168.0  ...  0.016346   
3        A       10686.0             5397.0       211.0  ...  0.019745   
4        A       10638.0             5191.0       195.0  ...  0.018331   

        roe   eps  book_value_per_share  num_analysts_covering  \
0  0.033470  0.52             15

In [23]:
merged['eps'].describe()

count    30853.000000
mean         1.239104
std          5.429596
min       -112.500000
25%          0.310000
50%          0.780000
75%          1.480000
max        654.220000
Name: eps, dtype: float64

In [25]:
merged.groupby(['dlycaldt']).size()

dlycaldt
2013-03-28    799
2013-06-28    800
2013-09-30    798
2013-12-31    798
2014-03-31    799
2014-06-30    799
2014-09-30    800
2014-12-31    798
2015-03-31    798
2015-06-30    796
2015-09-30    797
2015-12-31    794
2016-03-31    796
2016-06-30    796
2016-09-30    796
2016-12-30    795
2017-03-31    795
2017-06-30    794
2017-09-29    794
2017-12-29    795
2018-03-29    794
2018-06-29    795
2018-09-28    794
2018-12-31    793
2019-03-29    794
2019-06-28    793
2019-09-30    795
2019-12-31    794
2020-03-31    792
2020-06-30    794
2020-09-30    793
2020-12-31    796
2021-03-31    795
2021-06-30    796
2021-09-30    796
2021-12-31    795
2022-03-31    796
2022-06-30    797
2022-09-30    792
2022-12-30    797
2023-03-31    796
2023-06-30    796
2023-09-29    795
2023-12-29    797
dtype: int64

In [27]:
merged['in_sp500'].value_counts(normalize=True)

in_sp500
1    0.573346
0    0.426654
Name: proportion, dtype: float64

In [29]:
merged.shape

(35012, 24)

In [31]:
merged.columns

Index(['permno', 'dlycaldt', 'avg_volume_3m', 'stock_price', 'return_1m',
       'dlycap', 'ticker_x', 'total_assets', 'total_liabilities', 'net_income',
       'operating_income', 'cash_flow_from_operations', 'current_ratio',
       'debt_to_equity', 'roa', 'roe', 'eps', 'book_value_per_share',
       'num_analysts_covering', 'num_auditor_changes', 'num_restatements',
       'industry_name', 'ticker_y', 'in_sp500'],
      dtype='object')

In [33]:
merged.isnull().sum()

permno                           0
dlycaldt                         0
avg_volume_3m                    0
stock_price                      0
return_1m                        3
dlycap                           0
ticker_x                         0
total_assets                  4157
total_liabilities             4159
net_income                    4155
operating_income              6484
cash_flow_from_operations     4216
current_ratio                 9768
debt_to_equity                5516
roa                           4304
roe                           5524
eps                           4159
book_value_per_share          5531
num_analysts_covering            0
num_auditor_changes              0
num_restatements                 0
industry_name                    0
ticker_y                     14938
in_sp500                         0
dtype: int64

In [35]:
merged.shape

(35012, 24)

In [37]:
merged['dlycap'].describe()

count    3.501200e+04
mean     4.135320e+07
std      9.814544e+07
min      4.779216e+06
25%      1.128921e+07
50%      1.847524e+07
75%      3.719426e+07
max      3.035217e+09
Name: dlycap, dtype: float64

### Feature Engineering

In [42]:
# Drop duplicates and irrelevant columns
df = merged.drop_duplicates()
# df = df.sort_values(by='dlycaldt', ascending=True)
df['year'] = df['dlycaldt'].dt.year
df['month'] = df['dlycaldt'].dt.month
df['day'] = df['dlycaldt'].dt.day
df['day_of_week'] = df['dlycaldt'].dt.dayofweek
df['quarter'] = df['dlycaldt'].dt.quarter

# Create lagged variables for financial columns
financial_columns = ['avg_volume_3m',
 'return_1m',
 'total_assets',
 'total_liabilities',
    'dlycap',
 'net_income',
 'operating_income',
 'cash_flow_from_operations',
 'current_ratio',
 'debt_to_equity',
 'roa',
 'roe',
 'eps',
 'book_value_per_share',
 'num_analysts_covering',
 'num_auditor_changes',
 'num_restatements',
  'stock_price'                   
                         ] 

# Step 1: Calculate correlation matrix
correlation_matrix = df[financial_columns + ['in_sp500']].corr()

# Step 2: Find pairs of highly correlated features (corr > 0.7)
high_corr_pairs = [
    (col1, col2)
    for col1 in financial_columns
    for col2 in financial_columns
    if col1 != col2 and abs(correlation_matrix.loc[col1, col2]) > 0.7
]

# Step 3: Drop one feature in each high-correlation pair, keeping the one with higher correlation to the target
drop_features = set()
for col1, col2 in high_corr_pairs:
    if col1 not in drop_features and col2 not in drop_features:
        if abs(correlation_matrix.loc[col1, 'in_sp500']) > abs(correlation_matrix.loc[col2, 'in_sp500']):
            drop_features.add(col2)
        else:
            drop_features.add(col1)

# Convert to a list for dropping
drop_features = list(drop_features)

# Step 4: Drop features with high correlation from the DataFrame
financial_columns = [col for col in financial_columns if col not in drop_features]
df = df.drop(columns=drop_features)

# Sort by company identifier and date
df = df.sort_values(by=['permno', 'dlycaldt']).reset_index(drop=True)

# Missing values imputation
for col in financial_columns:
    df[col] = df.groupby('permno')[col].transform(
        lambda group: group.fillna(
            group.rolling(2, min_periods=1).mean()
        )
    )

for col in financial_columns:
    df[f'lag_{col}'] = df.groupby('permno')[col].shift(1)


# Calculate the growth factor of stock_price
df['growth_factor'] = (df['lag_stock_price']- df['lag_stock_price'].shift(1))/ df['lag_stock_price'].shift(1)

# Optional: Replace infinities or NaN resulting from division
df['growth_factor'] = df['growth_factor'].replace([float('inf'), -float('inf')], pd.NA).fillna(1)

# df = df.sort_values(by=['permno','year', 'month', 'day']).reset_index(drop=True)
df = df.sort_values(by=['dlycaldt']).reset_index(drop=True)
# Drop rows with missing lagged values if necessary
#df = df.drop(columns=["permno", "ticker_x", "ticker_y", "dlycaldt","quarter"])
df = df.drop(columns=["ticker_y", "dlycaldt","quarter"])
df = df.drop(columns=financial_columns)
df = df.dropna(subset=[f'lag_{col}' for col in financial_columns]).reset_index(drop=True)
df['Last_Quarter_Positive'] = df['lag_eps'] > 0
df=df.drop(columns=["lag_eps","lag_dlycap","lag_stock_price"])

In [43]:
df.head()

,permno,ticker_x,industry_name,in_sp500,year,month,day,day_of_week,lag_avg_volume_3m,lag_return_1m,...,lag_current_ratio,lag_debt_to_equity,lag_roa,lag_roe,lag_book_value_per_share,lag_num_analysts_covering,lag_num_auditor_changes,lag_num_restatements,growth_factor,Last_Quarter_Positive
0,90866,LBTYK,Transportation & Public Utilities,0,2013,6,28,4,1.219305e+06,1.945930,...,0.765140,24.665790,-0.000024,-0.000626,6.223072,0.0,0.0,0.0,1.0,False
1,90851,WPZ,Mining,0,2013,6,28,4,1.163957e+06,0.974659,...,0.693372,1.119590,0.016952,0.035961,23.111807,6.0,1.0,1.0,1.0,True
2,52090,MKC,Manufacturing,1,2013,6,28,4,7.505183e+05,1.238816,...,1.086060,1.383142,0.018600,0.044517,12.942746,0.0,0.0,0.0,1.0,True
3,92293,TDC,Technology,1,2013,6,28,4,1.907380e+06,2.200873,...,1.927203,0.721050,0.019556,0.033657,10.656535,5.0,0.0,0.0,1.0,True
4,92257,VMW,Technology,0,2013,6,28,4,3.259227e+06,0.407332,...,2.142244,0.806421,0.016250,0.029354,13.803119,5.0,0.0,0.0,1.0,True


In [44]:
df['growth_factor'].describe()

count    22870.000000
mean         0.071539
std          0.261899
min         -0.967420
25%         -0.051193
50%          0.034957
75%          0.127429
max          6.654532
Name: growth_factor, dtype: float64

In [45]:
df.head(20)

,permno,ticker_x,industry_name,in_sp500,year,month,day,day_of_week,lag_avg_volume_3m,lag_return_1m,...,lag_current_ratio,lag_debt_to_equity,lag_roa,lag_roe,lag_book_value_per_share,lag_num_analysts_covering,lag_num_auditor_changes,lag_num_restatements,growth_factor,Last_Quarter_Positive
0,90866,LBTYK,Transportation & Public Utilities,0,2013,6,28,4,1.219305e+06,1.945930,...,0.765140,24.665790,-0.000024,-0.000626,6.223072,0.0,0.0,0.0,1.0,False
1,90851,WPZ,Mining,0,2013,6,28,4,1.163957e+06,0.974659,...,0.693372,1.119590,0.016952,0.035961,23.111807,6.0,1.0,1.0,1.0,True
2,52090,MKC,Manufacturing,1,2013,6,28,4,7.505183e+05,1.238816,...,1.086060,1.383142,0.018600,0.044517,12.942746,0.0,0.0,0.0,1.0,True
3,92293,TDC,Technology,1,2013,6,28,4,1.907380e+06,2.200873,...,1.927203,0.721050,0.019556,0.033657,10.656535,5.0,0.0,0.0,1.0,True
4,92257,VMW,Technology,0,2013,6,28,4,3.259227e+06,0.407332,...,2.142244,0.806421,0.016250,0.029354,13.803119,5.0,0.0,0.0,1.0,True
5,86725,LIFE,Manufacturing,1,2013,6,28,4,2.129387e+06,0.795384,...,1.558619,0.822244,0.014193,0.025865,27.340713,0.0,0.0,0.0,1.0,True
6,90849,ROC,Manufacturing,0,2013,6,28,4,1.077720e+06,2.122347,...,3.322201,2.350889,0.003443,0.011897,19.263303,0.0,0.0,2.0,1.0,True
7,86745,BBRY,Technology,0,2013,6,28,4,6.674907e+07,-0.837509,...,2.059455,0.391649,0.007444,0.010359,18.363940,0.0,1.0,1.0,1.0,True
8,77178,QCOM,Technology,1,2013,6,28,4,1.168193e+07,0.389922,...,3.375641,0.265109,0.039203,0.049613,21.790846,7.0,0.0,1.0,1.0,True
9,86580,NVDA,Technology,1,2013,6,28,4,1.064954e+07,1.422925,...,4.891565,0.328219,0.027131,0.036036,7.827574,4.0,0.0,0.0,1.0,True


In [50]:
df['Last_Quarter_Positive'].value_counts()

Last_Quarter_Positive
True     19900
False     2970
Name: count, dtype: int64

In [52]:
df.columns

Index(['permno', 'ticker_x', 'industry_name', 'in_sp500', 'year', 'month',
       'day', 'day_of_week', 'lag_avg_volume_3m', 'lag_return_1m',
       'lag_total_assets', 'lag_operating_income',
       'lag_cash_flow_from_operations', 'lag_current_ratio',
       'lag_debt_to_equity', 'lag_roa', 'lag_roe', 'lag_book_value_per_share',
       'lag_num_analysts_covering', 'lag_num_auditor_changes',
       'lag_num_restatements', 'growth_factor', 'Last_Quarter_Positive'],
      dtype='object')

In [54]:
df.shape

(22870, 23)

In [56]:
df.head()

,permno,ticker_x,industry_name,in_sp500,year,month,day,day_of_week,lag_avg_volume_3m,lag_return_1m,...,lag_current_ratio,lag_debt_to_equity,lag_roa,lag_roe,lag_book_value_per_share,lag_num_analysts_covering,lag_num_auditor_changes,lag_num_restatements,growth_factor,Last_Quarter_Positive
0,90866,LBTYK,Transportation & Public Utilities,0,2013,6,28,4,1.219305e+06,1.945930,...,0.765140,24.665790,-0.000024,-0.000626,6.223072,0.0,0.0,0.0,1.0,False
1,90851,WPZ,Mining,0,2013,6,28,4,1.163957e+06,0.974659,...,0.693372,1.119590,0.016952,0.035961,23.111807,6.0,1.0,1.0,1.0,True
2,52090,MKC,Manufacturing,1,2013,6,28,4,7.505183e+05,1.238816,...,1.086060,1.383142,0.018600,0.044517,12.942746,0.0,0.0,0.0,1.0,True
3,92293,TDC,Technology,1,2013,6,28,4,1.907380e+06,2.200873,...,1.927203,0.721050,0.019556,0.033657,10.656535,5.0,0.0,0.0,1.0,True
4,92257,VMW,Technology,0,2013,6,28,4,3.259227e+06,0.407332,...,2.142244,0.806421,0.016250,0.029354,13.803119,5.0,0.0,0.0,1.0,True


In [58]:
# df[['permno','dlycaldt','total_assets','lag_total_assets']].head(10)

### ML Modeling

In [61]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Create a copy of the DataFrame
df_copy = df.copy()

# Ensure data is sorted by year and month
df_copy = df_copy.sort_values(by=['year', 'month']).reset_index(drop=True)

# Get unique combinations of year and month
unique_quarters = df_copy[['year', 'month']].drop_duplicates().reset_index(drop=True)

# Select the first 33 months for training, 6 months for validation, and remaining for testing
train_quarters = unique_quarters.iloc[:36]
validation_quarters = unique_quarters.iloc[36:41]
test_quarters = unique_quarters.iloc[41:]

# Create train, validation, and test datasets based on year and month
train_data = df_copy.merge(train_quarters, on=['year', 'month'], how='inner')
validation_data = df_copy.merge(validation_quarters, on=['year', 'month'], how='inner')
test_data = df_copy.merge(test_quarters, on=['year', 'month'], how='inner')

train_data = train_data.set_index(['permno', 'ticker_x'])
validation_data = validation_data.set_index(['permno', 'ticker_x'])
test_data = test_data.set_index(['permno', 'ticker_x'])

# Separate features (X) and target (y), keeping indices for tracking
X_train = train_data.drop(columns=['in_sp500'])
y_train = train_data['in_sp500']

X_validation = validation_data.drop(columns=['in_sp500'])
y_validation = validation_data['in_sp500']

X_test = test_data.drop(columns=['in_sp500'])
y_test = test_data['in_sp500']

# Identify categorical and numerical columns
categorical_features = ['industry_name', 'Last_Quarter_Positive']
numerical_features = [col for col in X_train.columns if col not in categorical_features and col not in ['year', 'month']]

# Fill missing values
X_train = X_train.fillna(0)
X_validation = X_validation.fillna(0)
X_test = X_test.fillna(0)

# Preprocessing pipeline
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

# Define models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=500),
    'Support Vector Machine': SVC(random_state=42, kernel='linear', probability=True),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=200, min_samples_split = 2),
}

def get_feature_names(preprocessor):
    """
    Extracts feature names after preprocessing
    """
    # Numerical features
    numeric_features = preprocessor.named_transformers_['num'].get_feature_names_out(numerical_features)
    
    # Categorical features (one-hot encoded)
    categorical_transformer = preprocessor.named_transformers_['cat']
    categorical_feature_names = categorical_transformer.get_feature_names_out(categorical_features)
    
    # Combine features
    return list(numeric_features) + list(categorical_feature_names)

def create_shap_plots(model, preprocessor, X_test, y_test, model_name):
    # Transform the test data
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names
    feature_names = get_feature_names(preprocessor)
    
    # Create SHAP explainer based on model type
    try:
        if isinstance(model, RandomForestClassifier):
            # For Random Forest
            explainer = shap.TreeExplainer(model)
            # Compute SHAP values
            shap_values = explainer.shap_values(X_test_transformed)
            shap_vals = shap_values[:, :, -1]
            print(f"Shape of X_test_transformed: {X_test_transformed.shape}")
            print(f"Shape of shap_vals: {shap_vals.shape}")
            # Summary plot
            plt.figure(figsize=(12, 8))
            shap.summary_plot(shap_vals, X_test_transformed, feature_names=feature_names, show=False)
            plt.title(f'SHAP Summary Plot - {model_name}')
            plt.tight_layout()
            plt.savefig(f'shap_summary_{model_name.replace(" ", "_")}.png')
            plt.close()
            
            # Bar plot
            plt.figure(figsize=(12, 8))
            shap.summary_plot(shap_vals, plot_type="bar", feature_names=feature_names, show=False)
            plt.title(f'SHAP Feature Importance - {model_name}')
            plt.tight_layout()
            plt.savefig(f'shap_importance_{model_name.replace(" ", "_")}.png')
            plt.close()
        
        elif isinstance(model, (LogisticRegression, SVC)):
            # For linear models
            explainer = shap.LinearExplainer(model, X_test_transformed)
            shap_values = explainer.shap_values(X_test_transformed)
            
            # Summary plot
            plt.figure(figsize=(12, 8))
            shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names, show=False)
            plt.title(f'SHAP Summary Plot - {model_name}')
            plt.tight_layout()
            plt.savefig(f'shap_summary_{model_name.replace(" ", "_")}.png')
            plt.close()
            
            # Bar plot
            plt.figure(figsize=(12, 8))
            shap.summary_plot(shap_values, plot_type="bar", feature_names=feature_names, show=False)
            plt.title(f'SHAP Feature Importance - {model_name}')
            plt.tight_layout()
            plt.savefig(f'shap_importance_{model_name.replace(" ", "_")}.png')
            plt.close()
    
        else:
            print(f"SHAP explanation not implemented for {model_name}")
    
    except Exception as e:
        print(f"Error creating SHAP plots for {model_name}: {e}")

# Train and evaluate each model with SHAP explanation
for model_name, classifier in models.items():
    # Build the pipeline
    model = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier),
        ]
    )

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on validation data
    y_pred_val = model.predict(X_validation)

    # Evaluate the model on validation data
    print(f"Model: {model_name}")
    print("Accuracy (validation):", accuracy_score(y_validation, y_pred_val))
    print("Classification Report (validation):\n", classification_report(y_validation, y_pred_val))

    # Make predictions on test data
    y_pred_test = model.predict(X_test)

    # Create a predictions DataFrame for easy joining
    predictions_df = pd.DataFrame({
        'year': X_test['year'],
        'month': X_test['month'],
        'predicted_in_sp500': y_pred_test,
        'actual_in_sp500': y_test
    }, index=X_test.index)

    # Evaluate the model on test data
    print("Accuracy (test):", accuracy_score(y_test, y_pred_test))
    print("Classification Report (test):\n", classification_report(y_test, y_pred_test))

    # Create SHAP plots
    create_shap_plots(model.named_steps['classifier'], 
                      model.named_steps['preprocessor'], 
                      X_test, 
                      y_test,
                      model_name)

    # Print a sample of the predictions
    print("Sample Predictions:\n", predictions_df.head(30))
    # print(predictions_df.loc[predictions_df.index.get_level_values('ticker_x') == 'FICO'])
    # print("-" * 50)

Model: Logistic Regression
Accuracy (validation): 0.7701149425287356
Classification Report (validation):
               precision    recall  f1-score   support

           0       0.76      0.35      0.48       766
           1       0.77      0.95      0.85      1757

    accuracy                           0.77      2523
   macro avg       0.77      0.65      0.67      2523
weighted avg       0.77      0.77      0.74      2523

Accuracy (test): 0.7819623389494549
Classification Report (test):
               precision    recall  f1-score   support

           0       0.79      0.38      0.51       304
           1       0.78      0.96      0.86       705

    accuracy                           0.78      1009
   macro avg       0.79      0.67      0.68      1009
weighted avg       0.78      0.78      0.75      1009

Sample Predictions:
                  year  month  predicted_in_sp500  actual_in_sp500
permno ticker_x                                                  
15488  PYPL      202

In [63]:
X_test.head(10)

,,industry_name,year,month,day,day_of_week,lag_avg_volume_3m,lag_return_1m,lag_total_assets,lag_operating_income,lag_cash_flow_from_operations,lag_current_ratio,lag_debt_to_equity,lag_roa,lag_roe,lag_book_value_per_share,lag_num_analysts_covering,lag_num_auditor_changes,lag_num_restatements,growth_factor,Last_Quarter_Positive
permno,ticker_x,,,,,,,,,,,,,,,,,,,,
15488,PYPL,Technology,2023,9,29,4,1.539759e+07,1.320984,74579.000,1426.000,970.000,1.296283,2.793246,0.013797,0.052337,17.841198,0.0,0.0,0.0,-0.121280,True
82515,POOL,Wholesale Trade,2023,9,29,4,3.910292e+05,1.084669,3680.579,336.584,376.777,2.717308,1.523482,0.063101,0.159235,37.351328,5.0,0.0,0.0,0.094031,True
13447,NOW,Technology,2023,9,29,4,1.522061e+06,2.534302,14923.000,235.000,1482.000,1.144963,1.154635,0.069959,0.150736,33.906603,0.0,0.0,1.0,0.209266,True
83639,CHKP,Technology,2023,9,29,4,8.753779e+05,-1.813350,5491.300,232.200,592.500,1.166392,0.942860,0.036785,0.071469,23.404713,8.0,0.0,0.0,-0.033692,True
12062,LH,Services,2023,9,29,4,6.112279e+05,0.353460,17718.700,466.300,472.600,2.345486,1.014673,0.010661,0.021503,99.041714,0.0,1.0,6.0,0.051914,True
64311,NSC,Transportation & Public Utilities,2023,9,29,4,1.514736e+06,0.345163,39261.000,1294.000,1846.000,0.680174,2.106092,0.009068,0.028165,55.679140,6.0,0.0,0.0,0.069623,True
83621,ANSS,Technology,2023,9,29,4,5.100141e+05,1.294280,6605.252,130.734,323.632,2.196573,0.344057,0.010526,0.014147,56.643100,6.0,0.0,0.0,-0.007602,True
87056,BMRN,Manufacturing,2023,9,29,4,1.293458e+06,-2.530080,6563.172,88.390,-3.870,4.939658,0.372237,0.008539,0.011717,25.420017,7.0,0.0,1.0,-0.108597,True
80864,RS,Wholesale Trade,2023,9,29,4,5.015004e+05,0.288025,10440.400,572.100,679.700,5.180529,0.369623,0.036886,0.050567,130.101134,0.0,0.0,0.0,0.057841,True


In [65]:
X_test.shape

(1009, 20)

In [67]:
X_train.shape

(19338, 20)